COMP 215 - LAB 3 Classes (NEO)
----------------
#### Name: Serena Harrington
#### Date: 20/01/2026

This lab exercise introduces `class` as a means of organizing related data and functions.

**Building on new concepts from lab 2**:
  * a `record` is a related collection of data, with fields for each data value
  * an `API` is an "Application Programmers Interface" defining how a programmer interacts with a system.
  * *f-string* simplifies string formatting operations

**New Python Concepts**:
  * the `class` keyword allows you define a new data `type`, with a set of operations on that data.
  * a `dataclass` simplifies class definition for classes that primarily encapsulate a data structure.

As usual, the first code cell simply imports all the modules we'll be using...

In [4]:
import datetime, json, requests
from datetime import datetime
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll continue working with [Near Earth Object](https://cneos.jpl.nasa.gov/) data
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

Here's a brief review from Lab 2 on how to use it...

### Review: making a query

Here's a query that gets the record for a single NEO that recently passed by.

In [5]:
API_KEY = 'DEMO_KEY'  # substitute your API key here

def get_neos(start_date):
    """ Return a list of NEO for the week starting at start_date """
    url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&api_key={API_KEY}'
    # Fetch last week's NEO feed
    response = requests.request("GET", url, headers={}, data={})
    data = json.loads(response.text)
    return [neo for dated_records in data['near_earth_objects'].values() for neo in dated_records ]

def get_neo(id):
    """ Return a NEO record for the given id """
    url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
    response = requests.request("GET", url, headers={}, data={})
    return json.loads(response.text)

# Sample usage:  get the list of NEOs for a given week, then lookup the latest NEO record in that list.
week_start = '2023-01-15'
neos = get_neos(week_start)
print(f'{len(neos)} Near Earth Objects found for week of {week_start}')
assert len(neos) > 0, f'Oh oh!  No NEOs found for {week_start}'
neo = get_neo(neos[-1]['id'])  # get the very latest NEO
pprint(neo)

13 Near Earth Objects found for week of 2023-01-15
{'absolute_magnitude_h': 27.9,
 'close_approach_data': [{'close_approach_date': '1963-10-03',
                          'close_approach_date_full': '1963-Oct-03 10:42',
                          'epoch_date_close_approach': -197126280000,
                          'miss_distance': {'astronomical': '0.0314084333',
                                            'kilometers': '4698634.721717071',
                                            'lunar': '12.2178805537',
                                            'miles': '2919596.2325987398'},
                          'orbiting_body': 'Venus',
                          'relative_velocity': {'kilometers_per_hour': '52882.6650360426',
                                                'kilometers_per_second': '14.6896291767',
                                                'miles_per_hour': '32859.2315053121'}},
                         {'close_approach_date': '1966-10-20',
                         

## Exercise 1: Define a CloseApproach class

Each NEO record comes with a list of `close_approach_data`, where each record in this list represents a single “close approach” to another orbiting body.
* Develop a class named `CloseApproach` to represent a *single* close approach record.
* State variables are
    * orbiting body (`str`)
    * approach date (`datetime` object!)
    * miss distance (`float` in km, document it!)
    * relative velocity (`float` in km/hr, ditto)

* Operations must include:
    * `__init__(self, ...)` method to initialize a new object with specific data values
    * `__str__(self)` method to return a nicely formatted string representation of the object.

Write a little code to test your new class.

In [6]:
class CloseApproach:
  """ CloseApproach class for representing a single close approach record """

  def __init__(self, orbiting_body:'str', approach_date:'datetime', miss_distance:'float', relative_velocity:'float'):
    """ Initialize a new object for given parameters with miss distance in km and relative velocty in km/hr """
    self.orbiting_body = orbiting_body
    self.approach_date = approach_date
    self.miss_distance = miss_distance
    self.relative_velocity = relative_velocity

  def __str__(self):
    """ Return a string representation of the close approach data """
    return f'orbiting body: {self.orbiting_body}, approach date: {self.approach_date}, miss distance: {self.miss_distance}, relative velocity: {self.relative_velocity}'


In [7]:
# Test CloseApproach class
ca = CloseApproach(orbiting_body = 'Mars', approach_date = '2200-05-24', miss_distance = 1495843.3943824, relative_velocity = 472847.2939285)

assert str(ca) == 'orbiting body: Mars, approach date: 2200-05-24, miss distance: 1495843.3943824, relative velocity: 472847.2939285'

## Exercise 2: Factory function: get_close_approach

We want to be able to construct CloseApproach objects easily from a data record returned from the NEO API.  

Write an "object factory" function...   

    def get_close_approach(record):
        ...

This function provides an easy way to create a `CloseApproach` instance.  It takes a dictionary for a single `close_approach_data` record, constructs and returns a `CloseApproach` object representing that same record.
This kind of function is called a “Factory” because it handles the details of constructing an object from raw materials.

Remember to convert each element from the data dictionary to the correct type (e.g., parse the date/time string into a `datetime` object).
Add little code to test your new factory function.

In [8]:
# Ex. 2 your code here
def get_close_approach(record:'dict'):
  """ Return a CloseApproach object for a single close approach data record """
  return CloseApproach(
      orbiting_body = record['orbiting_body'],
      approach_date = datetime.strptime(record['close_approach_date'], '%Y-%m-%d'),
      miss_distance = float(record['miss_distance']),
      relative_velocity = float(record['relative_velocity'])
  )

In [9]:
# Test get_close_approach function
d = {'orbiting_body': 'Earth', 'close_approach_date': '2100-03-28', 'miss_distance': '29829328.29382938', 'relative_velocity': '2382.93849383'}
assert get_close_approach(d).orbiting_body == 'Earth'

## Exercise 3:  Define an Asteroid class

Define a simple Asteroid class with some basic state variables representing a single NEO.  Your Asteroid class should define at least 4 "state variables:”
* id  (`int`)
* name (`str`)
* estimated_diameter (`float` in m)
* is_potentially_hazardous (`bool`)
* close_approaches (`list` of CloseApproach objects, default to empty list)

Operations must include:
* `__init__(self, ...)` method to initialize a new object with specific data values
* `__str__(self)` method to return a nicely formatted string representation of the object.

Write a little code to test you new class (just leave close_approaches as an empty list for now).

In [10]:
# Ex. 3 your code here
class Asteroid:
  """ Asteroid class for representing a single NEO """


  def __init__(self, id:'int', name:'str', estimated_diameter:'float', is_potentially_hazardous:'bool', close_approaches:'list' = []):
    self.id = id
    self.name = name
    self.estimated_diameter = estimated_diameter
    self.is_potentially_hazardous = is_potentially_hazardous
    self.close_approaches = close_approaches


  def __str__(self):
    return f"id: {self.id}, name: {self.name}, estimated diameter: {self.estimated_diameter}, is potentially hazardous: {self.is_potentially_hazardous}, close approaches: {self.close_approaches}"

  def get_closest_to_earth(self):
    """ Return CloseApproach object from asteroid representing its nearest to Earth """
    earth_objects = [neo for neo in self.close_approaches if neo.orbiting_body == 'Earth']
    closest_approach = earth_objects[0]
    for approach in earth_objects:
      if approach.miss_distance < closest_approach.miss_distance:
        closest_approach = approach
    return closest_approach


In [11]:
# Test Asteroid class
a = Asteroid(id = 19828293, name = 'Ceres', estimated_diameter = 239893829.2392938, is_potentially_hazardous = True, close_approaches = [])
assert str(a) == 'id: 19828293, name: Ceres, estimated diameter: 239893829.2392938, is potentially hazardous: True, close approaches: []'

## Exercise 4: Asteroid factory

Write a function that returns an Asteroid object just from the id for a single NEO.

    def asteroid_from_neo(neo_id):
        ...

This factory function takes the `id` for a single NEO, fetches the NEO record from API, constructs and returns an Asteroid object representing that NEO.  *Hint*: I provided the code fetch a NEO from its `id` above.

Every `Asteroid` should have a list of “close approaches”.
*Hint*: use the `get_close_approach` factory you defined above to construct the required list of CloseApproach objects.

Now add a new method to `Asteroid` class to return the `CloseApproach` object from the asteroid representing its nearest to **Earth**:

    def nearest_miss(self):
        ...

Extend your test code to demonstrate these new features.

In [15]:
# Ex. 4 your test code for nearest_miss here
def asteroid_from_neo(neo_id):
  """ Return an Asteroid object representing a single NEO from it's id """
  neo = get_neo(neo_id)

  return Asteroid(
      id = neo['id'],
      name = neo['name'],
      estimated_diameter = float(neo['estimated_diameter']['meters']['estimated_diameter_max']),
      is_potentially_hazardous = neo['is_potentially_hazardous'],
      close_approaches = neo['close_approach_data']
  )


In [13]:
asteroid_from_neo(3797409)

{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/3797409?api_key=DEMO_KEY'}, 'id': '3797409', 'neo_reference_id': '3797409', 'name': '(2018 AV)', 'designation': '2018 AV', 'nasa_jpl_url': 'https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=3797409', 'absolute_magnitude_h': 23.53, 'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.0523064176, 'estimated_diameter_max': 0.1169607054}, 'meters': {'estimated_diameter_min': 52.3064175804, 'estimated_diameter_max': 116.9607053693}, 'miles': {'estimated_diameter_min': 0.032501691, 'estimated_diameter_max': 0.0726759905}, 'feet': {'estimated_diameter_min': 171.6089870545, 'estimated_diameter_max': 383.7293606038}}, 'is_potentially_hazardous_asteroid': False, 'close_approach_data': [], 'orbital_data': {'orbit_id': '8', 'orbit_determination_date': '2022-12-26 07:02:28', 'first_observation_date': '2017-11-17', 'last_observation_date': '2022-12-25', 'data_arc_in_days': 1864, 'observations_used': 37, 'orbit_uncertainty': '1

## Challenge - Take your skills to the next level...
### Exercise 5: develop a useful analysis / data product

 With these data structures in place, we can now start answering all kinds of interesting questions about a single Asteroid or a set of Asteroids.  
Here’s a couple ideas to try:

* write a **function** named `most_dangerous_approach`, that takes a date range and returns a single “potentially hazardous” Asteroid object that makes the closest approach to Earth within that range.  Your algorithm will ultimately need to:
    * grab the list of NEO’s for the given date range;
    * use a list comprehension to build the list of Asteroid objects for the NEO’s returned
    * use a list comprehension to filter  potentially hazardous Asteroids only;
    * use a list comprehension to map each Asteroid to its  nearest_miss
    * apply Python’s min function to identify the Asteroid with the nearest_miss

You may want to decompose some of these steps into smaller functions.
* add a method to the Asteroid class, `estimated_mass`, that computes an estimate of the Asteroid’s mass based on its diameter.  This is a model – state your assumptions.
* add a method to the `CloseApproach` class, `impact_force`,  that estimates the force of impact if the Asteroid hit the orbiting object.  Again, this is a model, state your assumptions.

In [14]:
# Ex. 5 (challenge) your code here